# Lab Tutorial 04
### Learning Outcomes 
In this tutorial we will review and practice what we have learned so far and learn new stuff to be able to write a function for permutation testing + calculating p-values for hypothesis testing. <br>
We will be using StudentsPerformance dataset from the second lab tutorial.<br>
Again the dataset can be found in:
https://www.kaggle.com/spscientist/students-performance-in-exams


# Importing packages

In [1]:
# importing packages
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
%matplotlib inline

import numpy as np
import pandas as pd
# import scipy.stats as stats

# Load in the dataset

In [2]:
df = pd.read_csv('StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## A contingency table again!
_Question 1 - Create contingency table of ethnicity by gender

In [3]:
# a contingency table like before
## the entries in the table are numbers
## rows and columns are the unique categories in 'gender' and 'race/ethnicity' columns of the dataframe
ctab = pd.crosstab(df['race/ethnicity'], df['gender'], margins = True)
ctab 

gender,female,male,All
race/ethnicity,,,
group A,36,53,89
group B,104,86,190
group C,180,139,319
group D,129,133,262
group E,69,71,140
All,518,482,1000


In [4]:
## what is the probability of a student being from group A ethnicity?
### reminder: how to get entries from the contingency table
#### ********** the column name should come first **********
p_groupA = ctab['All']['group A']/ctab['All']['All']
p_groupA 

0.089

In [5]:
# using crosstab to get a table with true/false values (yes/no)
ctab2 = pd.crosstab(df['race/ethnicity'] == 'group A', df['gender'], margins = True)
ctab2 

gender,female,male,All
race/ethnicity,,,
False,482,429,911
True,36,53,89
All,518,482,1000


In [6]:
# Calculate the probability again
p_groupA2 = ctab2['All'][True]/ctab2['All']['All']
p_groupA2 

0.089

# Defining functions again!

## Write a function that takes in a dataframe and a column name and returns the mean and std of that column

In [11]:
df['math score'].mean()  

66.089

In [8]:
def my_function(data, colname): 
    # takes in pandas dataframe: data and a colname
    # returns the mean and std of the column

    mean_col = data[colname].mean()
    std_col  = data[colname].std()

    return (mean_col, std_col) 

### Use the function to get the mean and std of column "math score"

In [12]:
[m, s] = my_function(df, "math score")
print(f"the mean is {m}")
print(f"the std is {s}")

the mean is 66.089
the std is 15.16308009600945


## Write a function that takes in a dataframe and calculate the probability of all the races
Useful when you are doing this question _(What we are doing here is more complicated than what you need to do for the question)_ :<br>
Task 1/Question 3 - Define a function that calculates this difference in probability (10pts)

In [19]:
races = df.groupby('race/ethnicity').groups.keys()
print(races)

np.zeros((1, len(races)))

pd.crosstab(df['race/ethnicity'] == 'group A', df['gender'], margins = True)

dict_keys(['group A', 'group B', 'group C', 'group D', 'group E'])


gender,female,male,All
race/ethnicity,,,
False,482,429,911
True,36,53,89
All,518,482,1000


In [22]:
# you can use groupby to get all the races
# use crosstab to create a contingency table and calculate the probability 
# initialize an array to store the calculated probabilities
# sometimes you need to initialize multiple variables that are to be used in the loop

def prob_race(data, gvar = 'race/ethnicity'):
    # takes in dataframe, data, and calculate the probability of all the races

    ## use groupby to get all the race groups
    races = data.groupby(gvar).groups.keys()

    ## initialize the numpy array
    probs = np.zeros((1, len(races)))

    i = 0 # this will be the index I will use in the loop
    for r in races:
        print(r)
        ctab_tmp = pd.crosstab(df['race/ethnicity'] == r, df['gender'], margins = True)
        prob_tmp = ctab_tmp['All'][True]/ctab_tmp['All']['All']
        probs[0, i] = prob_tmp
        i = i + 1 

    return probs

### Use the funciton to calculate the probabilities

In [23]:
P = prob_race(df, gvar = 'race/ethnicity')
print(P) 

group A
group B
group C
group D
group E
[[0.089 0.19  0.319 0.262 0.14 ]]


## Write a function that takes the number of rows and columns of a numpy array and returns a numpy array with elements drawn from normal distribution (a random 2D numpy array)
This is just for practicing function. It is not usually the way we use to create a matrix of random numbers <br>
Use __np.random.normal__ : https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.normal.html

In [25]:
# Again: initializing numpy arrays and more on indexing
## setting default values for the inputs
## determine the dimensions of the matrix (numpy array) based on the inputs

def rand_array(nrow = 2, ncol = 2):
    # creates a nrow-by-ncol numpy array and fill it with numbers drawn from normal dist.

    out_nump = np.zeros((nrow, ncol)) # initialize with zeros

    for i in np.arange(nrow):
        for j in np.arange(ncol):
            out_nump[i, j] = np.random.normal(0, 1) # the default value for size is one.

    return out_nump

### Use the function to create a 3 by 3 array of random numbers

In [30]:
A = rand_array(nrow = 3, ncol = 3)
print(A)
A.shape

[[ 0.63062885 -1.20647633  1.43885376]
 [-0.31605576 -0.82512613 -1.08723204]
 [-0.16640315  1.15240897 -0.65103533]]


(3, 3)

### Print all the entries (with 3 decimal points) with row and col number:
the output should be like:<br>
"The entry on row 1 col 2 is 0.1"

In [27]:
# get the number of rows and columns first
[r, c] = A.shape

# use a for loop to access each entry
# use % for string formatting.
for i in np.arange(r):
    for j in np.arange(c):
        print("The entry on row %d col %d is %.3f" % (i, j, A[i, j]))

The entry on row 0 col 0 is 1.847
The entry on row 0 col 1 is -0.627
The entry on row 0 col 2 is 1.022
The entry on row 1 col 0 is 0.559
The entry on row 1 col 1 is -0.652
The entry on row 1 col 2 is -0.152
The entry on row 2 col 0 is 0.187
The entry on row 2 col 1 is -0.033
The entry on row 2 col 2 is -2.231


### Practice more:
https://www.w3resource.com/python-exercises/python-functions-exercises.php

# Problem Description
An increasingly common statistical tool for determining whether a difference is significant is the randomization or permutation test. A randomization test builds - rather than assumes – a distribution of the test statistics. This is achieved by exchanging or permuting variables which are “exchangeable” under the Null-hypothesis. A randomization test gives a simple way to compute the distribution for any test statistic under the null hypothesis that the dependent variable is statistically not dependent of the shuffled variable.<br><br><br>
Assuming that the gender variable is exchangeable, randomly shuffle the dataframe.<br>
The docs that you will need:
* https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.shuffle.html
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html


We want to test the hypothesis that male students are more likely than female students to take test preparation course. Under this hypothesis the p(test|male) should be higher than p(test|female).

## 1. create a copy of the dataframe
you don't want to change the original dataframe cause in each iteration, you want to shuffle the original one.

In [44]:
# first create a copy of your dataframe. You don't want to change the original dataframe while you are shuffling
# then use np.random.shuffle and df.reset_index

## 1. create a copy
dfcopy = df.copy()
dfcopy


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


## 2. create an array that represent the indices
as the indices are integers, you can create an array, having the same numbers as the indices in the dataframe and use it as the index!

In [45]:
## 2. create an array that represent the indices
inds = np.arange(len(df.index))
# print("before shuffling:")
print(inds)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

## 3. use np.random.shuffle to randomly shuffle indices.
the array that represents the indices will be shuffled.

In [46]:
## 3. use np.random.shuffle to randomly shuffle indices.
### use print to see the effect of shuffling
np.random.shuffle(inds)
# print("After shuffling")
print(inds)

[349 403 967  93 105 847 713 399 881 214 143 153 994  34 486 316 917 515
 400   3 603 870 926 239 299 637 580 362 711  90 629 884 401 442  77 565
 358 168 459 331 564 754 950 814 207 923 261 487 561  54 431 427 871 157
 785 407 254 623 329 854 708 507 170 757  64 513 548 521 968 370  87 802
 412 275 303 939 522 832 119 189 543 169 530 904 426 701  25 840 815 446
 438 661 415 123 925 174  80 534  20 432 919 348 625 979 777 700 668 672
 627 269 394 640 983 417 694 894 896  97 966 117 977 232 453 575 556 837
 602 843 537 402 378   1  67 746 818 540 990 955 930 238 585 492  22 405
 129 885 809 488 376 450 660 690 789 819 650 981 826 913  91  31 163 741
 782 825 497 199 529 323 250 593 333 659 991 544 140  94 217 573 900 195
 584 204 203 604 947 792 611 291 335  42 615  52 451 516 975 654 765 937
 571 985 652 961 245 721  10 324 980 353 617 350 467 695 339 294 798 539
 482  86 301 509 685 223 850 588 594 524 356 343  59 519 976 799 644 300
 869 172 905 205 251 876 107 274  41 535 982 237 63

## 4. shuffle the exchangeable variable
Which variable is exchangeable under the null hypothesis?

In [47]:
## 4. shuffle the exchangeable variable
tmp = df['gender'].iloc[inds]
tmp

349      male
403    female
967      male
93       male
105    female
        ...  
813      male
749      male
258    female
218      male
742    female
Name: gender, Length: 1000, dtype: object

## 5. reorder the shuffled indices. 
You will be putting the shuffled variable back to the dataframe. You want the indices for your dataframe to start from 0 (like the original one). You don't want to use the shuffled indices.<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.reset_index.html

In [48]:
### for index, we always want the numbers to go from 0 to ...
tmp = tmp.reset_index(drop = True)
tmp

0        male
1      female
2        male
3        male
4      female
        ...  
995      male
996      male
997    female
998      male
999    female
Name: gender, Length: 1000, dtype: object

## 6. put the shuffled exchangeable variable back into the copy of the dataframe

In [49]:
## 6. put the shuffled gender variable back into the copy of the dataframe
dfcopy['gender'] = tmp

## Now compare the original dataframe with the copy you created!

In [50]:
# Compare df and dfcopy
## only the exchangeable variable is shuffled.
display(df.head())
display(dfcopy.head())

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,male,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,female,group C,some college,standard,none,76,78,75


## Check to see if the proportion of each category of the exchangeable variable changed

In [51]:
# Checking the number of males and females before and after shuffling
cTab3 = pd.crosstab(df['race/ethnicity'], df['gender'], margins = True)
cTab4 = pd.crosstab(dfcopy['race/ethnicity'], dfcopy['gender'], margins = True)
display(cTab3)
display(cTab4)

gender,female,male,All
race/ethnicity,,,
group A,36,53,89
group B,104,86,190
group C,180,139,319
group D,129,133,262
group E,69,71,140
All,518,482,1000


gender,female,male,All
race/ethnicity,,,
group A,54,35,89
group B,94,96,190
group C,175,144,319
group D,130,132,262
group E,65,75,140
All,518,482,1000


## Another way of shuffling
How about creating a random array of 0s and 1s (0s for males, 1s for females)?<br>
you can create a random array of 0s and 1s using random.randint:<br>
https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.random.randint.html

In [52]:
# create an array with the same length as the index of the dataframe
shuff_var = np.random.randint(2, size=len(dfcopy.index))
shuff_var.shape

(1000,)

In [53]:
dfcopy['gender_2'] = shuff_var
display(dfcopy.head())

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,gender_2
0,male,group B,bachelor's degree,standard,none,72,72,74,0
1,female,group C,some college,standard,completed,69,90,88,0
2,male,group B,master's degree,standard,none,90,95,93,0
3,male,group A,associate's degree,free/reduced,none,47,57,44,0
4,female,group C,some college,standard,none,76,78,75,0


### Now check the proportions

In [54]:
# Checking the number of males and females before and after shuffling
cTab3 = pd.crosstab(df['race/ethnicity'], df['gender'], margins = True)
cTab4 = pd.crosstab(dfcopy['race/ethnicity'], dfcopy['gender_2'], margins = True)
display(cTab3)
display(cTab4)

gender,female,male,All
race/ethnicity,,,
group A,36,53,89
group B,104,86,190
group C,180,139,319
group D,129,133,262
group E,69,71,140
All,518,482,1000


gender_2,0,1,All
race/ethnicity,,,
group A,40,49,89
group B,97,93,190
group C,166,153,319
group D,153,109,262
group E,84,56,140
All,540,460,1000


## 7. Calculate the stat
After you have created successfully shuffled the exchangeable variable in the dataset, you can calculate the stat.

In [58]:
# Again use cross tab
ctab5 = pd.crosstab(dfcopy['test preparation course'] == 'completed', dfcopy['gender'], margins = True)
ctab5

gender,female,male,All
test preparation course,,,
False,331,311,642
True,187,171,358
All,518,482,1000


In [59]:
# calculate the probabilities and print out the values
p_com_fem = ctab5['female'][True]/ctab5['female']['All']
p_com_m   = ctab5['male'][True]/ctab5['male']['All']
print(f"probability that the student has taken test preparation course given that the student is female: {p_com_fem}")
print(f"probability that the student has taken test preparation course given that the student is male: {p_com_m}")


probability that the student has taken test preparation course given that the student is female: 0.361003861003861
probability that the student has taken test preparation course given that the student is male: 0.35477178423236516


# All the steps above must be done in a loop (same as bootstrap)
like the bootstrap, you will have to perform the steps above within a loop with a specific number of iterations! You will end up with a stat for each iteration!<br>
Once the loop is done, you can calculate the p-value: 
* __One Sided:__ How many times the calculated stats were bigger than the actual value of the stat, calculated from the original data?/ number of times you have permuted the exchangeable variable.

### What you need to do to calculate the p-value?
The calculations you need to do will be similar to what we will do in this example:<br>
Suppose you have created an array of 1000 random integers between 0 and 100. What is the probability that created integers is higher than 60?

In [64]:
# First I'm going to create an array of 1000 random integers between 0 and 100.
## This is just an example to show you how you can caulculate the probability.
## you don't need to create an array of random numbers for the task.
Ex_array = np.random.randint(101, size=1000)
#print(Ex_array)

# reminder: Boolean operations!
### determine which numbers are >= 60
bool_b_60 = Ex_array >= 60
#print(bool_b_60)

# Calculate the probability
nB60 = sum(bool_b_60)
pB60 = nB60/1000
pB60

0.416